<a href="https://colab.research.google.com/github/Lokkamithran/FYP/blob/main/bi_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Bidirectional
from numpy.random import seed
import matplotlib.pyplot as plt

seed(42)
tf.random.set_seed(42)

In [ ]:
vocab_size = 100000

In [ ]:
!pwd
import os
os.chdir('/content/gdrive/My Drive/FYP_Colab/Bi-LSTM')
!pwd

/content
/content/gdrive/My Drive/FYP_Colab/Bi-LSTM


In [ ]:
df = pd.read_csv("../Final.csv", sep="`")

In [ ]:
df = df.dropna()
# df = df.sample(frac=1).reset_index(drop=True)

#independent features
x = df["Comment"]
#dependent feature
y = df["Fine Tag"]

In [ ]:
encoding = [one_hot(words,vocab_size) for words in x]
emb_doc = pad_sequences(encoding, padding='pre', maxlen=100)
# print(emb_doc)

import pickle
with open("test.pkl", "wb") as f:
    pickle.dump(one_hot, f)

with open("test.pkl", "rb") as f:
    encoder = pickle.load(f)
new_encoding =[encoder(d, vocab_size) for d in df.text]

In [ ]:
final_x = np.array(emb_doc)
final_y = np.array(y)

# print(type(y), " ", type(final_y))

# x_train, x_test, y_train, y_test = train_test_split(final_x, final_y, test_size=0.2, random_state=42)

<class 'pandas.core.series.Series'>   <class 'numpy.ndarray'>


In [ ]:
performance_df = pd.DataFrame(columns=['Batch Size','Train-Test Split','Epochs','Training Acc.','Testing Acc.','Precision','Recall','F1 Score','F0.5 Score'])
epochsList = [2, 3, 4, 5]
ttSplitList = [0.2, 0.3]
batchSizeList = [32, 64, 128]

In [ ]:
for currentBatchSize in batchSizeList:
    for currentTTSplit in ttSplitList:
        for currentNumberOfEpochs in epochsList:

            model = Sequential()
            model.add(Embedding(vocab_size, 50, input_length=100))
            model.add(Bidirectional(LSTM(100)))
            model.add(Dense(1, activation='sigmoid'))
            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

            x_train, x_test, y_train, y_test = train_test_split(final_x, final_y, test_size = currentTTSplit, random_state=42)
            history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = currentNumberOfEpochs, batch_size = currentBatchSize)

            acc_train = history.history['binary_accuracy']
            acc_val = history.history['val_binary_accuracy']

            predictions = model.predict(x_test)
            new_list = []
            for i in range(0,predictions.size):
                value = predictions[i][0]
                new_list.append(0 if (value<0.5) else 1)

            tn, fp, fn, tp = confusion_matrix(y_test, new_list).ravel()

            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            f1_score = 2/((1/precision)+(1/recall))
            f05_score = 1.25/((0.25/recall)+(1/precision))

            performance_df.loc[len(performance_df.index)] = [currentBatchSize, currentTTSplit, currentNumberOfEpochs, round(acc_train[-1], 3),
                                                             round(acc_val[-1], 3), round(precision, 3), round(recall, 3), round(f1_score, 3), round(f05_score, 3)]

performance_df.to_excel(excel_writer=r'Performance_Metrics.xlsx', index=False)

In [ ]:
noOfEpochs = 2
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=noOfEpochs, batch_size=50)


# Tried modifying:

# 1. Epochs
# 2. Batch size
# 3. Number of Dense layers
# 4. LSTM neurons
# 5. max length of comments
# 6. word embedding dimensions
# 7. Vocabulary size

# Max possible accuracy (test) ~ 68%

In [ ]:
epochs = range(1,noOfEpochs+1)

acc_train = history.history['binary_accuracy']
acc_val = history.history['val_binary_accuracy']
plt.plot(epochs, acc_train, 'g', label='Training accuracy')
plt.plot(epochs, acc_val, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
!ls

bi-lstm.pkl  encoder.pkl


In [ ]:
pickled_model = pickle.load(open('bi-lstm.pkl', 'rb'))
with open('encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

In [ ]:
# model.save('the_bi_lstm_model.keras')
# !cp -r '/content/the_bi_lstm_model.keras' /content/gdrive/MyDrive/FYP_Colab/
# loaded_model = tf.keras.models.load_model('gdrive/MyDrive/FYP_Colab/the_bi_lstm_model_new.keras')

sample_text = "nee laam uyir vaala ve kudathu"
test_encoding = [one_hot(sample_text, vocab_size)]
test_emb_doc = pad_sequences(test_encoding, padding="pre", maxlen=100)

predictions = model.predict(np.array(test_emb_doc))
print(predictions[0][0])

# Print the label based on the prediction
if predictions[0] < 0.5:
    print('The comment is positive')
else:
    print('The comment is negative')

In [ ]:
#Prediction variable
predictions = model.predict(x_test)
new_list = []
for i in range(0,predictions.size):
    value = predictions[i][0]
    new_list.append(0 if (value<0.5) else 1)

tn, fp, fn, tp = confusion_matrix(y_test, new_list).ravel()

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = 2/((1/precision)+(1/recall))
f05_score = 1.25/((0.25/recall)+(1/precision))


print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)
print("F0.5 score: ", f05_score)
# print(y_test)
# print(np.array(new_list))

In [ ]:
print("Accuracy: ", accuracy_score(y_test, new_list))